In [1]:
import time 
import logging
import os
import sqlite3
import pathlib
import glob
import gzip 

import pandas as pd
import xmltodict

DOSSIER_PARENT = "."
DOSSIER_SOURCE = "./todo/"
DOSSIER_SORTIE = "./done/"
BDD = 'bdd_actes_budgetaires_gz.db'
NOM_CSV = 'donnees_budgetaires.csv'


In [2]:

logging.basicConfig(level=logging.INFO,
                    filename='traitement.log', 
                    filemode='w')

def timing_decorator(func):
 def wrapper(*args, **kwargs):
  start_time = time.time()
  result = func(*args, **kwargs)
  end_time = time.time()
  execution_time = end_time - start_time
  print(f"{func.__name__} a pris {execution_time:.4f} secondes pour s'exécuter.")
  return result
 return wrapper

def parsing_fichier(chemin) : 
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def _extract_ligne_budget(data_dict: dict) -> pd.DataFrame : 
 "Sépare les sous clefs lignes budget, sans nettoyage"
 ligne_budget = data_dict['DocumentBudgetaire']['Budget']['LigneBudget']
 df_ligne_budget = pd.DataFrame(ligne_budget)
 return df_ligne_budget

def _extract_metadonnees(data_dict : dict) -> pd.DataFrame : 
 "Sépare les sous clefs de métadonnées, sans nettoyage"
 metadonnees = data_dict['DocumentBudgetaire']['EnTeteDocBudgetaire']
 df_metadonnees = pd.DataFrame(metadonnees)
 return df_metadonnees

def _extract_id(fichier) : 
 "Extrait l'id du nom du fichier"
 val_IdFichier = fichier.split("-")[-1].split('.')[0]
 IdFichier = {"IdFichier" : val_IdFichier}
 df_id = pd.DataFrame(IdFichier, index = [0])
 return df_id

def _extract_date(data_dict : dict) -> pd.DataFrame : 
 date = data_dict['DocumentBudgetaire']['Budget']['BlocBudget']
 df_date = pd.DataFrame(date)
 return df_date

def extraction_donnees(data_dict : dict, fichier) : 
 df_ligne_budget =_extract_ligne_budget(data_dict)
 df_metadonnees = _extract_metadonnees(data_dict)
 df_date = _extract_date(data_dict)
 df_id = _extract_id(fichier)
 return df_ligne_budget, df_metadonnees, df_date, df_id

def assemblage(df_principal: pd.DataFrame, 
                         df_meta: pd.DataFrame, 
                         df_id: pd.DataFrame,
                         df_date : pd.DataFrame) -> pd.DataFrame:
 """ Assemble les dataFrame contenant les metadonnees,
 la version schema et les lignes budgetaires """
 colonnes_a_conserver = ["IdFichier", "DteDec", "LibelleColl",
                         "IdColl","Nature","LibCpte",
                         "Fonction","Operation",
                         "ContNat","ArtSpe",
                         "ContFon", "ContOp",
                         "CodRD","MtBudgPrec",
                         "MtRARPrec","MtPropNouv",
                         "MtPrev","CredOuv",
                         "MtReal","MtRAR3112",
                         "OpBudg","TypOpBudg",
                         "OpeCpteTiers"
                        ]
 
 df_final = pd.DataFrame(columns=colonnes_a_conserver)
 df_schem = pd.concat([df_id] * len(df_principal), ignore_index=True)
 df_meta = pd.concat([df_meta] * len(df_principal), ignore_index=True)
 df_date = pd.concat([df_date] * len(df_principal), ignore_index=True)

 for col in df_id.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_schem[col]

 for col in df_meta.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_meta[col]

 for col in df_principal.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_principal[col]

 for col in df_date.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_date[col]

 df_final = df_final.dropna(axis=1, how='all')
 return df_final

def nettoyage_lambda(df : pd.DataFrame) -> pd.DataFrame : 
 "Nettoie les données pour se débarasser des @V"
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in df.columns : 
  df[col] = df[col].apply(nettoyage)
 return df 

def insertion_bdd(df_final: pd.DataFrame):
 """ insert dans une bdd les données maintenant transformées et en sort un csv à jour """
 chemin_bdd = os.path.join(DOSSIER_PARENT, BDD)
 conn = sqlite3.connect(chemin_bdd)
 df_final.to_sql('acte_budgetaire_gz', conn,
                    if_exists='append', index=False)
 
def creation_csv() :
 conn = sqlite3.connect(BDD)
 cursor = conn.cursor()
 info = cursor.execute('''Select * from acte_budgetaire_gz''')
 df = pd.DataFrame(info)
 fichier_csv = os.path.join(DOSSIER_PARENT, NOM_CSV)
 df.to_csv(fichier_csv, index=False)
 conn.close()

def deplacement_fichier(fichier_a_deplacer, dossier_destination):
 """ Déplace le fichier du dossier source au dossier fini"""
 chemin_source = pathlib.Path(fichier_a_deplacer)
 chemin_destination = pathlib.Path(dossier_destination) / chemin_source.name
 chemin_source.rename(chemin_destination)

def clean_bdd_test() : 
 conn = sqlite3.connect(BDD)
 cursor = conn.cursor()
 cursor.execute('''DELETE FROM acte_budgetaire_gz ''')
 conn.commit()
 conn.close()

def supr_csv_test(dossier = DOSSIER_PARENT, nom_fichier = NOM_CSV) :
 fichier_a_supprimer = os.path.join(dossier, nom_fichier)
 os.remove(fichier_a_supprimer)

@timing_decorator
def main():
 """ Traitement global, extrait et transforme les fichiers XML dans DOSSIER_SOURCE
    pour les insérer dans une bdd et en faire un csv"""
 liste_des_fichier_source = glob.glob(os.path.join(DOSSIER_SOURCE, "*.gz"))
 liste_des_fichier_traite = glob.glob(os.path.join(DOSSIER_SORTIE, "*.gz"))
 liste_des_df = []
 for fichier in liste_des_fichier_source:
  logging.info(f'Debut du travail sur {fichier}')
    # Sécurité permettant de ne pas injecter des doublons
  if fichier in liste_des_fichier_traite : 
   logging.error(f'Le fichier {fichier} a déjà été traité')
   return None 
  else : 
   data_dict = parsing_fichier(fichier)
   if data_dict is not None:
    df_budget = _extract_ligne_budget(data_dict)
    df_metadonnees = _extract_metadonnees(data_dict)
    df_id = _extract_id(fichier)
    df_date = _extract_date(data_dict)
    df_final = assemblage(
                df_budget, df_metadonnees, df_id, df_date)
    liste_des_df.append(df_final)
    logging.info(f'Fin du travail sur {fichier}')
 df_session = pd.concat(liste_des_df, ignore_index = True)
 nettoyage_lambda(df_session)
 insertion_bdd(df_session)
 creation_csv()
 clean_bdd_test()



In [10]:
for _ in range(10):
    main()

main a pris 6.1959 secondes pour s'exécuter.
main a pris 6.5505 secondes pour s'exécuter.
main a pris 6.8662 secondes pour s'exécuter.
main a pris 6.8334 secondes pour s'exécuter.
main a pris 7.2359 secondes pour s'exécuter.
main a pris 7.4756 secondes pour s'exécuter.
main a pris 7.4985 secondes pour s'exécuter.
main a pris 7.5651 secondes pour s'exécuter.
main a pris 8.0656 secondes pour s'exécuter.
main a pris 8.1442 secondes pour s'exécuter.


In [9]:
liste_des_fichier_source = glob.glob(os.path.join(DOSSIER_SOURCE, "*.gz"))
liste_des_fichier_traite = glob.glob(os.path.join(DOSSIER_SORTIE, "*.gz"))

def parse_id(fichier) : 
 val_IdFichier = fichier.split("-")[-1].split('.')[0]
 return val_IdFichier

def test_comprehension():
    conn = sqlite3.connect(BDD)
    cursor = conn.cursor()
    cursor.execute(''' SELECT DISTINCT IdFichier FROM acte_budgetaire_gz ''')
    ligne_sql_int = [int(x[0]) for x in cursor.fetchall()]
    return ligne_sql_int

files_to_print = [fichier for fichier in liste_des_fichier_source if int(parse_id(fichier)) not in test_comprehension()]


# Parcourez la liste des fichiers à imprimer
def main_safe()
for fichier in files_to_print:
    print(f"{fichier} vous bisou")



In [27]:
from lxml import etree 

def parse_fichier_xml(chemin) -> dict:
    """ Transforme le fichier xml en dictionnaire python """
    with open(chemin, "r", encoding="latin-1") as fichier_ouvert:
        fichier_xml = fichier_ouvert.read()
        data_dict = xmltodict.parse(fichier_xml)
    return data_dict

way_autrexml = "./autre_xml/ppal-bp-2021.xml"

arbre = etree.parse(way_autrexml)
racine = arbre.getroot()
enfants = racine.getchildren()
attributs = racine.attrib
text = racine.text
enfants 


[<Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}VersionSchema at 0x22f8b889180>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}BlocEditeur at 0x22f8247e880>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}VersionOutil at 0x22f8b8f9c40>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}VersionOutil at 0x22f8b8d62c0>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Scellement at 0x22f8b8d4f40>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}EnTeteDocBudgetaire at 0x22f8b8d7dc0>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Budget at 0x22f8b8d7c40>]

In [60]:
num_enfant = 6 
entete = enfants[num_enfant]
arbreentete = etree.ElementTree(entete)
larbreentete = arbreentete.getroot()
attribentete = arbreentete.getroot().attrib
textarbre = arbreentete.getroot().text
larbreentete.getchildren()


[<Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}EnTeteBudget at 0x22f8b8d3600>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}BlocBudget at 0x22f8c953000>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}InformationsGenerales at 0x22f8b8b1cc0>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f824f4480>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f8c996bc0>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f8c9942c0>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f8c994900>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f8c996740>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f8c994240>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f8c995000>,
 <Element {http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget at 0x22f

In [82]:

lignes_budget_list = []

# Parcourez les enfants de larbreentete
for caca in larbreentete.getchildren():
    # Vérifiez si l'enfant est de type "lignes budget"
    if caca.tag == "{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget":
        # Créez un dictionnaire pour stocker les informations de cette ligne budget
        ligne_budget_dict = {}
        
        # Accédez aux informations à l'intérieur de l'enfant "lignes budget"
        for sous_element in caca.getchildren():
            # Utilisez le nom de la balise comme clé et le texte à l'intérieur comme valeur
            ligne_budget_dict[sous_element.tag] = sous_element.attrib
        
        # Ajoutez le dictionnaire de cette ligne budget à la liste
        lignes_budget_list.append(ligne_budget_dict)


In [83]:
lignes_budget_list

[{'{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Nature': {'V': '60611'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Fonction': {'V': '90'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}ContNat': {'V': '011'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}ArtSpe': {'V': 'false'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}CodRD': {'V': 'D'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtBudgPrec': {'V': '5500.00'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtPropNouv': {'V': '8500.00'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtPrev': {'V': '8500.00'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}OpBudg': {'V': '0'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtSup': {'Code': 'BudgetHorsRAR', 'V': '5500.00'},
  '{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}CaracSup': {'Code': 'Section', 'V': 'F'}},
 {'{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Nature': {'V': '60611'},
  '{http://www.mi

In [87]:
lignes_budget_list2 = []

for ligne_budget_element in larbreentete.iter("{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LigneBudget"):
    # Créez un dictionnaire pour stocker les informations de cette ligne budget
    ligne_budget_dict = {}
    
    # Parcourez les enfants de l'élément "LigneBudget"
    for sous_element in ligne_budget_element.getchildren():
        # Utilisez le nom de la balise comme clé et le texte à l'intérieur comme valeur
        ligne_budget_dict[sous_element.tag] = sous_element.attrib["V"]
    
    # Ajoutez le dictionnaire de cette ligne budget à la liste
    lignes_budget_list2.append(ligne_budget_dict)

lignes_budget_list2

df_bbb = pd.DataFrame(lignes_budget_list2)
df_bbb

,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Nature,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Fonction,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}ContNat,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}ArtSpe,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}CodRD,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtBudgPrec,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtPropNouv,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtPrev,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}OpBudg,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}MtSup,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}CaracSup,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}TypOpBudg,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}LibCpte,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}OpeCpteTiers,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}Operation,{http://www.minefi.gouv.fr/cp/demat/docbudgetaire}ContOp
0,60611,90,011,false,D,5500.00,8500.00,8500.00,0,5500.00,F,NaN,NaN,NaN,NaN,NaN
1,60611,251,011,false,D,1000.00,750.00,750.00,0,1000.00,F,NaN,NaN,NaN,NaN,NaN
2,60611,324,011,false,D,500.00,500.00,500.00,0,500.00,F,NaN,NaN,NaN,NaN,NaN
3,60611,322,011,false,D,1210.00,1210.00,1210.00,0,1210.00,F,NaN,NaN,NaN,NaN,NaN
4,60611,311,011,false,D,450.00,450.00,450.00,0,450.00,F,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,1313,414,NaN,false,R,655146.00,157795.40,157795.40,0,631181.00,I,NaN,EQUIPEMENTS STRUCTURANTS ONET,NaN,113,NaN
858,13141,414,NaN,false,R,2733739.00,546747.80,546747.80,0,2186991.20,I,NaN,EQUIPEMENTS STRUCTURANTS ONET,NaN,113,NaN
859,1317,414,NaN,false,R,1300000.00,1300000.00,1300000.00,0,1300000.00,I,NaN,EQUIPEMENTS STRUCTURANTS ONET,NaN,113,NaN
860,2313,414,NaN,false,D,2847981.00,624363.56,624363.56,0,624363.56,I,NaN,EQUIPEMENTS STRUCTURANTS ONET,NaN,113,113
